In [1]:
# import libraries
import BioSimSpace as BSS
import os
import sys
import glob
import csv
import numpy as np
import networkx as nx
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
from sklearn.preprocessing import minmax_scale, MinMaxScaler
import itertools as it


print("adding code to the pythonpath...")
code = "/home/anna/Documents/code/python"
if code not in sys.path:
    sys.path.insert(1, code)
import pipeline

from pipeline.prep import *
from pipeline.utils import *
from pipeline.setup import initialise_pipeline

pipeline.__file__

import random
import math
import pandas as pd
import subprocess

fwf_path = (
    "/home/anna/Documents/september_2022_workshops/freenrgworkflows/networkanalysis"
)
if fwf_path not in sys.path:
    sys.path.insert(1, fwf_path)

import networkanalysis

from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics import mean_squared_error as mse

from scipy.stats import sem as sem
from scipy.stats import bootstrap, norm
from scipy.stats import spearmanr

[Errno 2] No such file or directory: '/home/anna/anaconda3/envs/biosimspace-dev/bin/pmemd.cuda'
could not determine AMBER version.


Warning on use of the timeseries module: If the inherent timescales of the system are long compared to those being analyzed, this statistical inefficiency may be an underestimate.  The estimate presumes the use of many statistically independent samples.  Tests should be performed to assess whether this condition is satisfied.   Be cautious in the interpretation of the data.
/home/anna/anaconda3/envs/biosimspace-dev/lib/python3.9/site-packages/numpy/core/getlimits.py:500: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/anna/anaconda3/envs/biosimspace-dev/lib/python3.9/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
/home/anna/anaconda3/envs/biosimspace-dev/lib/python3.9/site-packages/numpy/core/getlimits.py:500: UserWarning: The value of the smallest 


Sending anonymous Sire usage statistics to http://siremol.org.
For more information, see http://siremol.org/analytics
To disable, set the environment variable 'SIRE_DONT_PHONEHOME' to 1
To see the information sent, set the environment variable 
SIRE_VERBOSE_PHONEHOME equal to 1. To silence this message, set
the environment variable SIRE_SILENT_PHONEHOME to 1.



adding code to the pythonpath...


/home/anna/anaconda3/envs/biosimspace-dev/lib/python3.9/site-packages/Bio/pairwise2.py:278: BiopythonDeprecationWarning: Bio.pairwise2 has been deprecated, and we intend to remove it in a future release of Biopython. As an alternative, please consider using Bio.Align.PairwiseAligner as a replacement, and contact the Biopython developers if you still need the Bio.pairwise2 module.
  warnings.warn(


In [ ]:
protein = "tyk2"

exec_folder = f"/home/anna/Documents/benchmark/{protein}_benchmark/execution_model"

##### <span style="color:teal">Comparing lomap and the rbfenn</span>  


In [ ]:
# get a list of the perts in each and all together
perts_lomap = []
perts_rbfenn = []
perts = []

file_a = f"{exec_folder}/network_lomap.dat"
# file_a = f"a-optimal-lomap.csv"
file_b = f"{exec_folder}/network_rbfenn.dat"
# file_b = f"d-optimal-lomap.csv"

perts_rbfenn, ligs = pipeline.analysis.get_info_network(file_a)
perts_lomap, ligs = pipeline.analysis.get_info_network(file_b)
perts = perts_rbfenn + perts_lomap

# write a file that contains the combined perts, directions are distinct
combined_perts = []
filtered_out = 0
for pert in perts:
    if not pert in combined_perts:
        combined_perts.append(pert)
    else:
        filtered_out += 1
print(
    f"Removed {filtered_out} duplicate perts between lomap and rbfenn to give {len(combined_perts)} combined perts."
)

# write a file that contains the unique perts, 1 direction only.
filtered_perts = []
filtered_out = 0
for pert in combined_perts:
    inv_pert = pert.split("~")[1] + "~" + pert.split("~")[0]

    if not pert in filtered_perts and not inv_pert in filtered_perts:
        filtered_perts.append(pert)
    else:
        filtered_out += 1
print(
    f"Removed {filtered_out} inverse perts to give {len(filtered_perts)} unique perts, one direction only."
)

# get the perts that are unique to each
unique_perts = []
unique_out_lomap = 0
unique_out_rbfenn = 0
shared_out = 0

for pert in perts_lomap:
    inv_pert = pert.split("~")[1] + "~" + pert.split("~")[0]

    if not pert in perts_rbfenn and not inv_pert in perts_rbfenn:
        unique_perts.append((pert, "lomap"))
        unique_out_lomap += 1

for pert in perts_rbfenn:
    inv_pert = pert.split("~")[1] + "~" + pert.split("~")[0]

    if not pert in perts_lomap and not inv_pert in perts_lomap:
        unique_perts.append((pert, "rbfenn"))
        unique_out_rbfenn += 1

for pert in combined_perts:
    inv_pert = pert.split("~")[1] + "~" + pert.split("~")[0]

    if pert in perts_lomap or inv_pert in perts_lomap:
        if pert in perts_rbfenn or inv_pert in perts_rbfenn:
            unique_perts.append((pert, "shared"))
            shared_out += 1


print(
    f"There are {unique_out_lomap} pert(s) unique to lomap and {unique_out_rbfenn} pert(s) unique to rbfenn."
)
print(f"There are {shared_out} pert(s) shared between lomap and rbfenn")

In [ ]:
# manually combined all perts,
perts_all, ligs = pipeline.analysis.get_info_network("tyk2_all_perts.dat")
print(len(perts_all))
# print(perts_all)

all_perts = []

for pert in perts_all:
    inv_pert = pert.split("~")[1] + "~" + pert.split("~")[0]

    if pert in combined_perts or inv_pert in combined_perts:
        all_perts.append((pert, "old"))
    else:
        all_perts.append((pert, "new"))

df = pd.DataFrame(all_perts)

df.loc[df[1] == "new"]

In [ ]:
with open(f"{exec_folder}/combined_perts.dat", "w") as writefile:
    writer = csv.writer(writefile)
    for pert in combined_perts:
        writer.writerow([pert])
print(f"Total number of combined perturbations: {len(combined_perts)}")

with open(f"{exec_folder}/filtered_perts.dat", "w") as writefile:
    writer = csv.writer(writefile)
    for pert in filtered_perts:
        writer.writerow([pert])
print(f"Total number of filtered perturbations: {len(filtered_perts)}")

# write a file for the different perts
with open(f"{exec_folder}/unique_perts.dat", "w") as writefile:
    writer = csv.writer(writefile)
    for pert in unique_perts:
        writer.writerow([pert[0], pert[1]])
print(
    f"Total number of unique perturbations: {len(unique_perts)} (lomap: {unique_out_lomap}, rbfenn: {unique_out_rbfenn})"
)
print(f"Total number of shared perturbations: {shared_out}")

In [ ]:
# make a dicitonary of the perts for plotting the nx graph
both_pert_networks_dict = {}

for pert in filtered_perts:
    inv_pert = pert.split("~")[1] + "~" + pert.split("~")[0]

    if pert in perts_lomap and pert in perts_rbfenn:
        both_pert_networks_dict[pert] = "both"
    elif inv_pert in perts_lomap and pert in perts_rbfenn:
        both_pert_networks_dict[pert] = "both"
    elif pert in perts_lomap and pert not in perts_rbfenn:
        both_pert_networks_dict[pert] = "lomap"
    elif inv_pert in perts_lomap and pert not in perts_rbfenn:
        both_pert_networks_dict[pert] = "lomap"
    elif pert not in perts_lomap and pert in perts_rbfenn:
        both_pert_networks_dict[pert] = "rbfenn"
    elif inv_pert not in perts_lomap and pert in perts_rbfenn:
        both_pert_networks_dict[pert] = "rbfenn"

# create dict for images for the nx graph
image_dict = {}
# list files in inputs
input_files_for_image = sorted(
    os.listdir(f"{exec_folder}/visualise_network_lomap/inputs")
)
for in_file in input_files_for_image:
    lig_name_list = in_file.split("_")[1:]
    lig_name = "_".join(lig_name_list).split(".")[0]
    lig_number = in_file.split("_")[0]
    image_dict[lig_name] = lig_number

In [ ]:
both_pert_networks_dict

In [ ]:
# Generate the graph.
graph = nx.Graph()

# Loop over the nligands and add as nodes to the graph.
for lig in ligand_names:
    img = f"{exec_folder}/visualise_network_lomap/images/{image_dict[lig]}.png"
    graph.add_node(lig, image=img, label=lig, labelloc="t")

# make a dict of colours
# navy teal  #CC00CC
# clear is '#FF000000'
colour_dict = {"both": "navy", "lomap": "teal", "rbfenn": "hotpink"}

# Loop over the edges in the dictionary and add to the graph.
for edge in both_pert_networks_dict:
    graph.add_edge(
        edge.split("~")[0],
        edge.split("~")[1],
        color=colour_dict[both_pert_networks_dict[edge]],
    )

# Plot the networkX graph.
pos = nx.kamada_kawai_layout(graph)
colours = nx.get_edge_attributes(graph, "color").values()

plt.figure(figsize=(12, 12), dpi=150)
nx.draw(
    graph,
    pos,
    edge_color=colours,
    width=1,
    linewidths=5,
    node_size=2000,
    node_color="skyblue",
    font_size=12,
    labels={node: node for node in graph.nodes()},
)

plt.savefig(f"{exec_folder}/compared_networks_no_images.png", dpi=300)
# plt.show()

# Convert to a dot graph.
dot_graph = nx.drawing.nx_pydot.to_pydot(graph)

# Write to a PNG.
network_plot = f"{exec_folder}/compared_networks.png"
dot_graph.write_png(network_plot)

# Create a plot of the network.
img = mpimg.imread(network_plot)
plt.figure(figsize=(10, 10))
plt.axis("off")
plt.imshow(img)

In [ ]:
# calculate the lomap score for the combined network file (unfiltered)
combined_pert_network_dict = {}

for pert in combined_perts:
    lig_a = pert.split("~")[0]
    lig_b = pert.split("~")[1]
    # then, we need to find this index for our chosen edge that we are adding.
    lig_a_index = ligand_names.index(lig_a)
    lig_b_index = ligand_names.index(lig_b)
    # finally, we need to calculate this single lomap score.
    single_transformation, single_lomap_score = BSS.Align.generateNetwork(
        [ligands[lig_a_index], ligands[lig_b_index]],
        names=[ligand_names[lig_a_index], ligand_names[lig_b_index]],
        plot_network=False,
    )
    print(f"LOMAP score for {lig_a} to {lig_b} is {single_lomap_score[0]} .")
    combined_pert_network_dict[(lig_a, lig_b)] = single_lomap_score[0]

In [ ]:
# write the combined to a different network file

with open(f"{exec_folder}/network_combined.dat", "w") as network_file:
    writer = csv.writer(network_file, delimiter=" ")

    for pert, score in combined_pert_network_dict.items():
        # # based on the provided (at top of notebook) lambda allocations and LOMAP threshold, decide allocation.
        # if score == None or score < float(node.getInput("LOMAP Threshold")):
        #     num_lambda = node.getInput("DiffLambdaWindows")
        # else:
        #     num_lambda = node.getInput("LambdaWindows")

        num_lambda = node.getInput("LambdaWindows")

        # given the number of allocated lambda windows, generate an array for parsing downstream.
        lam_array_np = np.around(np.linspace(0, 1, int(num_lambda)), decimals=5)

        # make the array into a format readable by bash.
        lam_array = (
            str(lam_array_np)
            .replace("[ ", "")
            .replace("]", "")
            .replace("  ", ",")
            .replace("\n", "")
        )

        # write out both directions for this perturbation.
        if engine == "ALL":
            for eng in BSS.FreeEnergy.engines():
                writer.writerow([pert[0], pert[1], len(lam_array_np), lam_array, eng])
        else:
            writer.writerow([pert[0], pert[1], len(lam_array_np), lam_array, engine])
        # writer.writerow([pert[1], pert[0], len(lam_array_np), lam_array, engine])

### generate a and d optimal networks

code in other_workflows.
this has 4 files
exp_dg - this is just experimental dGs
pairwise_fep - this is the results from the fep (A, D, lit)
literature designed pairs - this is the pairs from literature and their fep ddG
ap_matrix - this is the matrix of the ligands with their similarity.

scripts are in R

In [ ]:
protein = "p38"

exec_folder = f"/home/anna/Documents/benchmark/{protein}_benchmark/execution_model"

In [ ]:
# to generate a network, need to make matrix as for cdk2

links_file = f"{exec_folder}/links_file.in"

# rbfenn file

value_dict = {}

# need to normalise the rbfenn so less close to 0 as otherwise is not able to det(matrix) way more than 0 when using the R script.

with open(links_file, "r") as f:
    for line in f:
        lig0 = line.split(" ")[0].strip()
        lig1 = line.split(" ")[1].strip()
        score = line.split(" ")[2].strip()
        if lig0 not in value_dict.keys():
            value_dict[lig0] = {}
            value_dict[lig0][lig1] = score
        else:
            value_dict[lig0][lig1] = score

pd.options.display.float_format = "{:,.5f}".format
df = pd.DataFrame(value_dict).fillna(1)

df = df.sort_index()
df = df.reindex(sorted(df.columns), axis=1)

df.to_csv(
    f"{exec_folder}/{protein}_rbfenn_matrix.csv",
    index_label="ID",
)

In [ ]:
# lomap file

value_dict = {}

# initialise pipeline at top of file
pl = initialise_pipeline()
# where the ligands for the pipeline are located. These should all be in the same folder in sdf format
pl.ligands_folder(f"/home/anna/Documents/benchmark/inputs/{protein}/ligands")
# where the pipeline should be made
pl.main_folder("/home/anna/Documents/code/write/test")
pl.setup_ligands()

for lig0, lig1 in it.product(pl.ligand_names, pl.ligand_names):
    if lig0 != lig1:
        single_transformation, score = BSS.Align.generateNetwork(
            [pl.ligands_dict[lig0], pl.ligands_dict[lig1]],
            names=[lig0, lig1],
            plot_network=False,
            links_file=None,
        )
        if lig0 not in value_dict.keys():
            value_dict[lig0] = {}
            value_dict[lig0][lig1] = score[0]
        else:
            value_dict[lig0][lig1] = score[0]

df = pd.DataFrame(value_dict).fillna(1)

df.to_csv(f"{exec_folder}/{protein}_lomap_matrix.csv", index_label="ID")

### expected performance of lomap vs rbfenn

run in python script already

In [ ]:
protein = "mcl1"

exec_folder = f"/home/anna/Documents/benchmark/{protein}_benchmark/execution_model"

In [ ]:
# make best fit dict
best_fit_dict = {}

for name in [
    "lomap",
    "rbfenn",
    "lomap-a-optimal",
    "rbfenn-a-optimal",
    "lomap-d-optimal",
    "rbfenn-d-optimal",
]:
    df = pd.read_csv(f"{exec_folder}/network_{name}_stats_20.csv", header=None)

    x = np.array(df[0])
    no_bins = abs(round(math.sqrt(len(x))))

    # Fit a normal distribution to the data, mean and standard deviation
    mu, std = norm.fit(x)

    # plot histogram
    plt.hist(
        x, bins=no_bins, density=True, alpha=0.7, color="skyblue", edgecolor="grey"
    )

    # Plot the PDF.
    xmin, xmax = plt.xlim()

    mu, std = norm.fit(x)
    x = np.linspace(xmin, xmax, 100)
    y = norm.pdf(x, mu, std)

    best_fit_dict[name] = ((x, y), mu, std)

In [ ]:
col_dict = {}

for key in best_fit_dict.keys():
    hexadecimal_alphabets = "0123456789ABCDEF"
    color = "#" + "".join([random.choice(hexadecimal_alphabets) for j in range(6)])
    col_dict[key] = color

In [ ]:
# plot the differnet neworks

# plot the distributions
fig, ax = plt.subplots(figsize=(10, 10))

lines = []
mu_std_string = ""

for name in best_fit_dict.keys():
    col = col_dict[name]
    plt.plot(
        best_fit_dict[name][0][0],
        best_fit_dict[name][0][1],
        "k",
        linewidth=2,
        color=col,
    )
    lines += plt.plot(0, 0, c=col, label=name)
    mu_std_string += f"\n{name} : mu = {best_fit_dict[name][1]:.3f} , std = {best_fit_dict[name][2]:.3f}"

    ax.axvspan(
        best_fit_dict[name][1] + best_fit_dict[name][2],
        best_fit_dict[name][1] - best_fit_dict[name][2],
        color=col,
        alpha=0.2,
    )
labels = [l.get_label() for l in lines]
plt.legend(lines, labels, loc="upper right")

plt.xlabel("Error")
plt.ylabel("Frequency")
plt.title(f"{mu_std_string}")
plt.savefig(
    f"dist_errors_networks.png",
    dpi=300,
    bbox_inches="tight",
)
plt.show()